<a href="https://colab.research.google.com/github/BigTMiami/AdaptOrDie/blob/main/Amazon_Domain_Training_Loss_By_Batch_Size_and_Trained_Model_Download_Experiment_Version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Summary
This experiment checks two things - does the downloaded pre-trained model produce the same eval loss and does different batch size on eval change the loss.

# Details
* No training is done, only evaluation, so train dataset is only used for evaluation

# Results



# Setup

In [1]:
!pip install datasets
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.3/297.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 82.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━

In [3]:
from datasets import load_dataset
dataset = load_dataset("BigTMiami/amazon_25M_500_000_condensed")
dataset

Generating train split:   0%|          | 0/83949 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8340 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 83949
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 8340
    })
})

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
block_size = tokenizer.model_max_length
print(f"block_size:{block_size}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

block_size:512


# Train

In [5]:
from transformers import AutoConfig, AutoModelForMaskedLM

# config = AutoConfig.from_pretrained("BigTMiami/amazon_pretraining_500k_reviews")
# model = AutoModelForMaskedLM.from_config(config)
model = AutoModelForMaskedLM.from_pretrained("BigTMiami/amazon_pretraining_500k_checkpoints")

config.json:   0%|          | 0.00/672 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [6]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="amazon_25M_simple_5_000_batch_test",
    learning_rate=0.0005, # Paper for DAPT training
    #per_device_train_batch_size=34, NOT TRAINING
    per_device_eval_batch_size=32, #
    num_train_epochs=1, # 1 pass, 12k steps, 25 million reviews
    weight_decay=0.01,
    warmup_ratio=0.06, # Paper: warmup proportion of 0.06
    adam_epsilon=1e-6, # Paper 1e-6 (huggingface default 1e-08)
    adam_beta1=0.9, # Paper: Adam weights 0.9
    adam_beta2=0.98, # Paper: Adam weights 0.98 (huggingface default  0.999)
    lr_scheduler_type="linear",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    # load_best_model_at_end=True,
    # push_to_hub=True,
)

In [7]:
!pip install pynvml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 2.1 MB/s eta 0:00:00


In [8]:
from pynvml import *

def print_gpu_utilization():
    nvmlInit()
    handle = nvmlDeviceGetHandleByIndex(0)
    info = nvmlDeviceGetMemoryInfo(handle)
    gpu_used = info.used//1024**2
    print(f"GPU {gpu_used} MB")

In [9]:
print_gpu_utilization()

GPU 451 MB


In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm_probability=0.15
)

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
print_gpu_utilization()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


GPU 1399 MB


In [11]:
eval_results = trainer.evaluate()
batch_size_loss = eval_results["eval_loss"]
print_gpu_utilization()
print(f"Batch Size 32 Loss: {batch_size_loss:.2f}")
print(eval_results)

GPU 8875 MB
Batch Size 32 Loss: 7.07
{'eval_loss': 7.070096015930176, 'eval_runtime': 88.7204, 'eval_samples_per_second': 94.003, 'eval_steps_per_second': 2.942}


In [12]:
training_args.per_device_eval_batch_size=1 # Try batch size of 1

In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=dataset["validation"],
    data_collator=data_collator,
)
print_gpu_utilization()

GPU 8875 MB


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
eval_results = trainer.evaluate()
print_gpu_utilization()
batch_size_loss = eval_results["eval_loss"]
print(f"Batch Size 1 Loss: {batch_size_loss:.2f}")
print(eval_results)

GPU 8875 MB
Batch Size 1 Loss: 7.06
{'eval_loss': 7.06497859954834, 'eval_runtime': 633.0894, 'eval_samples_per_second': 13.173, 'eval_steps_per_second': 13.173}


In [15]:
print("Disconnecting Session")
from google.colab import runtime
runtime.unassign()

Disconnecting Session
